In [10]:
import pandas as pd 
import time
import numpy as np 

daily_stock_price_columns = ['index', 'time', 'code', 'open', 'close', 'low', 'high', 'volume', 'money', 'factor',
                            'high_limit', 'low_limit', 'avg', 'pre_close', 'paused', 'open_interest']
daily_stock_price_column_dtype = {
    'index': float, 'time': str, 'code': str, 'open': float, 'close': float, 'low': float, 'high': float,
    'volume': float, 'money': float, 'factor': float, 'high_limit': float, 'low_limit': float, 'avg': float, 
    'pre_close': float, 'paused': str, 'open_interest': str
}

df = pd.read_csv('./20200101-20201016.csv', header=None, parse_dates=['time'], na_values='\\N',
                 names=daily_stock_price_columns, dtype=daily_stock_price_column_dtype)

df['pct_change'] = (df.close - df.pre_close)/df.close * 100
df['daily_status'] = df['pct_change'].map(lambda x: 0 if x < 0 else 1)

daily_stock_price_columns_normalized = df.groupby('code').apply(
    lambda x: x.sort_values(by='time', ascending=True)[['open', 'close', 'low', 'high', 'volume', 'money', 'daily_status']].values)
print(daily_stock_price_columns_normalized)

# build the model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import Input

input_layer = Input(shape=(100, 6, 1))
x = Conv2D(filters=32, kernel_size=(16, 6), strides=(1, 6))(input_layer)
x = MaxPooling2D(pool_size=(10, 6), strides=(1, 6))(x)
x = Flatten(x)
output_layer = Dense(2, activation='softmax')

model = keras.Model(inputs=input_layer, outputs=output_layer, name='stock_model')
model.summary()

model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'],
             optimizer=keras.optimizer.Adam(lr=0.01, decay=0.9))


 


code
000001.XSHE    [[16.37, 16.58, 16.27, 16.66, 155657388.0, 257...
000002.XSHE    [[32.8, 32.56, 32.51, 33.6, 101213040.0, 33423...
000004.XSHE    [[22.65, 22.33, 22.17, 22.9, 1785320.0, 401449...
000005.XSHE    [[3.11, 3.14, 3.11, 3.15, 10413412.0, 32661410...
000006.XSHE    [[5.27, 5.24, 5.21, 5.29, 12855775.0, 67452865...
                                     ...                        
688596.XSHG    [[nan, nan, nan, nan, nan, nan, 1.0], [nan, na...
688598.XSHG    [[nan, nan, nan, nan, nan, nan, 1.0], [nan, na...
688599.XSHG    [[nan, nan, nan, nan, nan, nan, 1.0], [nan, na...
688600.XSHG    [[nan, nan, nan, nan, nan, nan, 1.0], [nan, na...
688981.XSHG    [[78.3, 78.63, 77.77, 82.89, 161873271.0, 1297...
Length: 4147, dtype: object


ValueError: Negative dimension size caused by subtracting 6 from 1 for '{{node max_pooling2d/MaxPool}} = MaxPool[T=DT_FLOAT, data_format="NHWC", ksize=[1, 10, 6, 1], padding="VALID", strides=[1, 1, 6, 1]](conv2d/BiasAdd)' with input shapes: [?,85,1,32].